# Taller inicial de Julia

## Environments and the Package manger (Pkg)

In [3]:
using Pkg
Pkg.activate(".")
Pkg.add("Parameters")
Pkg.add("Unitful")
Pkg.add("UnitfulAstro")
Pkg.add("Zygote")
Pkg.add("DataFrames")
Pkg.add("DifferentialEquations")

  Activating project at `~/casa/work/2023/TallerJulia`


    Updating registry at `~/.julia/registries/General.toml`


   Resolving package versions...


  No Changes to `~/casa/work/2023/TallerJulia/Project.toml`
  No Changes to `~/casa/work/2023/TallerJulia/Manifest.toml`


   Resolving package versions...


...

  No Changes to `~/casa/work/2023/TallerJulia/Project.toml`
  No Changes to `~/casa/work/2023/TallerJulia/Manifest.toml`


   Resolving package versions...


  No Changes to `~/casa/work/2023/TallerJulia/Project.toml`
  No Changes to `~/casa/work/2023/TallerJulia/Manifest.toml`


   Resolving package versions...


  No Changes to `~/casa/work/2023/TallerJulia/Project.toml`
  No Changes to `~/casa/work/2023/TallerJulia/Manifest.toml`


## Arrays

In [4]:
# Arrays
a = [1.0, 2.0, 3.0]
@show a typeof(a)
@show typeof(a)<:Vector{Integer}
@show typeof(a)<:Vector{Float64};


a = [1.0, 2.0, 3.0]
typeof(a) = Vector{Float64}
typeof(a) <: Vector{Integer} = false
typeof(a) <: Vector{Float64} = true


In [ ]:
a[begin]==a[1]  # a[end]==a[3]

In [ ]:
@show sum(a)
b = 2a
@show c = a + b
@show c = a + b.*b
@show c = a .+ b'b;

In [ ]:
m = [a 2a 3a]
display(m)
display(m')

## Structs

In [ ]:
struct Evento{T<:Real,D<:Real}
    t::T
    x::D
    v::D
end

In [ ]:
a = Evento(0, 10., 1.)

In [ ]:
a = Evento(0, 10.) # show what happens

In [ ]:
Evento(x::T,y::T) where {T<:Real} = Evento(0., x, y)

In [ ]:
Evento(10., 1.)

In [ ]:
methods(Evento)

In [ ]:
Evento(; t=9, x=7, v) = Evento(t, x, v)

In [ ]:
a = Evento(t=0, x=1, v=1)

In [ ]:
b = Evento(t=98)

In [ ]:
b = Evento(v=54)

## Parameters package

In [10]:
using Parameters
@with_kw struct Event{T<:Real,D<:Real}
    t::T = 0.0
    x::D = 5.0
    v::D
end

Event

In [11]:
methods(Event)

# 5 methods for type constructor:
 [1] Event(; t, x, v)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:545
 [2] Event(pp::Event; kws...)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:569
 [3] Event(pp::Event, di::AbstractDict)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:572
 [4] Event(pp::Event, di::Tuple{Symbol, Any}...)
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:573
 [5] Event(t::T, x::D, v::D) where {T<:Real, D<:Real}
     @ ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:526

In [15]:
b = Event(v=3) # then Event(v=3.0)

MethodError: MethodError: no method matching Event(::Float64, ::Float64, ::Int64)

Closest candidates are:
  Event(::T, ::D, !Matched::D) where {T<:Real, D<:Real}
   @ Main ~/.julia/packages/Parameters/MK0O4/src/Parameters.jl:526


## Unitful Quantities

In [26]:
using Unitful, UnitfulAstro
x = rand(3)u"kpc"
v = rand(3)u"km/s"
t = 10^6*u"s"
@show x v t

x = Quantity{Float64, 𝐋, Unitful.FreeUnits{(kpc,), 𝐋, nothing}}[0.9299809364900019 kpc, 0.7222774396075209 kpc, 0.19684742803206134 kpc]
v = Quantity{Float64, 𝐋 𝐓^-1, Unitful.FreeUnits{(km, s^-1), 𝐋 𝐓^-1, nothing}}[0.961336404523244 km s^-1, 0.2177251470073298 km s^-1, 0.0808422578624699 km s^-1]
t = 1000000 s


1000000 s

In [27]:
a = v/t

3-element Vector{Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-2), 𝐋 𝐓^-2, nothing}}}:
  9.61336404523244e-7 km s^-2
 2.177251470073298e-7 km s^-2
 8.084225786246991e-8 km s^-2

In [30]:
uconvert.(u"km/s/Myr",a)

3-element Vector{Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}}:
 3.0337469719382722e7 km s^-1 Myr^-1
  6.870883099198511e6 km s^-1 Myr^-1
   2.55118763672068e6 km s^-1 Myr^-1

In [31]:
u"G"

6.6743e-11 m^3 kg^-1 s^-2

In [34]:
G = uconvert(u"kpc*Msun^-1*(km/s)^2",u"G")

4.30091727003628e-6 km^2 kpc s^-2 M⊙^-1

## Automatic Differentiation

In [35]:
using Zygote
f(x,y) = exp(x/y)*cos(3x+2y)

f (generic function with 1 method)

In [49]:
∇f(x,y) = gradient((x,y)->f(x,y), x,y)

∇f (generic function with 1 method)

In [51]:
∇f(1.,2.)

(-2.6280759982354986, -2.4771192304363403)

In [52]:
∇f(x) = gradient(x->f(x...), x)

∇f (generic function with 2 methods)

In [53]:
methods(∇f)

# 2 methods for generic function "∇f" from Main:
 [1] ∇f(x)
     @ ~/casa/work/2023/TallerJulia/diciembre2023.ipynb:1
 [2] ∇f(x, y)
     @ ~/casa/work/2023/TallerJulia/diciembre2023.ipynb:1

In [64]:
@show ∇f([1.,2.])[1]  ∇f(1.,2.)

(∇f([1.0, 2.0]))[1] = [-2.6280759982354986, -2.4771192304363403]
∇f(1.0, 2.0) = (-2.6280759982354986, -2.4771192304363403)


(-2.6280759982354986, -2.4771192304363403)

In [74]:
f(x,y) = exp(x)*exp(7y)
Df(x,y) = [1,7]*f(x,y)
∇f(x,y) = gradient((x,y)->f(x,y), x,y)
df(x,y) = gradient(f, x,y)

df (generic function with 1 method)

In [78]:
x = [5.,-7.]
@show Df(x...) ∇f(x...) df(x...)
@show Df(x...) .== ∇f(x...);
@show Df(x...) .== df(x...);

Df(x...) = [7.781132241133797e-20, 5.446792568793657e-19]
∇f(x...) = (7.781132241133797e-20, 5.446792568793657e-19)
df(x...) = (7.781132241133797e-20, 5.446792568793657e-19)
Df(x...) .== ∇f(x...) = Bool[1, 1]
Df(x...) .== df(x...) = Bool[1, 1]


## Data Frames

In [89]:
using DataFrames
n=20
data = DataFrame(id=collect(1:n), x=rand(n), y=rand(n), z=rand(n))

Row,id,x,y,z
,Int64,Float64,Float64,Float64
1,1,0.771163,0.935211,0.317131
2,2,0.615179,0.347024,0.711141
3,3,0.803694,0.638969,0.563208
4,4,0.929355,0.864594,0.709465
5,5,0.0766896,0.434191,0.546728
6,6,0.407404,0.919991,0.0466591
7,7,0.911533,0.455475,0.814988
8,8,0.748734,0.599982,0.294086
9,9,0.779079,0.0112006,0.823612


In [90]:
deleteat!(data, [3,5,7])

Row,id,x,y,z
,Int64,Float64,Float64,Float64
1,1,0.771163,0.935211,0.317131
2,2,0.615179,0.347024,0.711141
3,4,0.929355,0.864594,0.709465
4,6,0.407404,0.919991,0.0466591
5,8,0.748734,0.599982,0.294086
6,9,0.779079,0.0112006,0.823612
7,10,0.0921933,0.77871,0.875819
8,11,0.114836,0.656846,0.475584
9,12,0.299873,0.644964,0.159851


## Galactic Dynamics 

In [91]:
using GalacticDynamics

In [102]:
pot = Plummer(34u"Msun", 10u"pc")
pot₂ = NFW(10^3*u"Msun", 5u"kpc")
@show pot pot₂ pot+pot₂

pot = Plummer{Int64, Rational{Int64}}
  m: Int64 34
  a: Rational{Int64}

pot₂ = 

NFW{Int64, Int64}
  m: Int64 1000
  a: Int64 5



pot + pot₂ = GalacticDynamics.AbstractPotential[Plummer{Int64, Rational{Int64}}
  m: Int64 34
  a: Rational{Int64}
, NFW{Int64, Int64}
  m: Int64 1000
  a: Int64 5
]


2-element Vector{GalacticDynamics.AbstractPotential}:
 Plummer{Int64, Rational{Int64}}
  m: Int64 34
  a: Rational{Int64}

 NFW{Int64, Int64}
  m: Int64 1000
  a: Int64 5


In [108]:
x=[5.0, 1.0, -2.0]u"pc"
@show potential(pot,x)
@show acceleration(pot,x)
@show potential(pot₂,x)
@show acceleration(pot₂,x)
@show potential(pot+pot₂,x)
@show acceleration(pot+pot₂,x);

potential(pot, x) = -0.01282532353900881 km^2 s^-2
acceleration(pot, x) = Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.4932816745772619 km s^-1 Myr^-1, -0.09865633491545238 km s^-1 Myr^-1, 0.19731266983090476 km s^-1 Myr^-1]
potential(pot₂, x) = -1.0202395888647748 km^2 s^-2
acceleration(pot₂, x) = Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.09304976990451902 km s^-1 Myr^-1, -0.018609953980903805 km s^-1 Myr^-1, 0.03721990796180761 km s^-1 Myr^-1]
potential(pot + pot₂, x) = Quantity{Float64, 𝐋^2 𝐓^-2, Unitful.FreeUnits{(km^2, s^-2), 𝐋^2 𝐓^-2, nothing}}[-1.0330649124037836 km^2 s^-2, -1.0330649124037836 km^2 s^-2, -1.0330649124037836 km^2 s^-2]
acceleration(pot + pot₂, x) = Quantity{Float64, 𝐋 𝐓^-2, Unitful.FreeUnits{(km, s^-1, Myr^-1), 𝐋 𝐓^-2, nothing}}[-0.5863314444817809 km s^-1 Myr^-1, -0.11726628889635618 km s^-1 Myr^-1, 0.23453257779271236 km s^-1 Myr^-1]


### DifferentialEquations.jl https://docs.sciml.ai/DiffEqDocs/stable/solvers/ode_solve/

In [127]:
using DifferentialEquations
using Test

function example_orbit()
    m_gal = 2.325e7*u"Msun"
    m =1018.0*m_gal  # Msun
    a = 2.562*u"kpc"     # kpc
    Λ = 200.0*u"kpc"    # kpc
    γ = 2.0
    pot = AllenSantillanHalo(m, a, Λ, γ)
    for i in range(1,20)
        w₀ = 50*rand(6)
        x₀ = w₀[1:3]u"kpc"
        v₀ = w₀[4:6]u"km/s"
        t_range = (0.0,100.0).*𝕦.t
        sol = evolve(pot, x₀, v₀, t_range)
        sol₂ = evolve(pot, x₀, v₀, t_range; options=SolverConfig(solver=Tsit5(), reltol=5.0e-12))
        @show @test sol.x[end] ≈ sol₂.x[end] rtol=5.0e-8
        @show @test sol.v[end] ≈ sol₂.v[end] rtol=5.0e-8
    end
end

example_orbit (generic function with 1 method)

In [128]:
example_orbit()

#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


...

#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed


#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:18 =# @test(sol.x[end] ≈ sol₂.x[end], rtol = 5.0e-8) = Test Passed
#= /home/mmestre/casa/work/2023/TallerJulia/diciembre2023.ipynb:19 =# @test(sol.v[end] ≈ sol₂.v[end], rtol = 5.0e-8) = Test Passed
